In [1]:
# Parameters
cpu = 8
group_name = "Astro-TE_NN"
mem_gb = 10


In [2]:
import pandas as pd
import subprocess
import numpy as np

In [3]:
#group_name = 'DG_Glut'

In [4]:
home_dir = '/home/qzeng_salk_edu/project/241018_pre_ml'
ct = group_name

dmr_dir = '/data/mC_FILTER_DMR_BED'
_type = 'Total'

In [5]:
dmr_bf = pd.read_csv(f'{dmr_dir}/{ct}.{_type}.aDMR.bed', sep='\t', header=None)
dmr_bf.head()

,0,1,2,3,4,5,6
0,chr1,3805471,3805536,chr1-3805471-3805536,0.80,0.90,0.62
1,chr1,4253018,4253156,chr1-4253018-4253156,0.65,0.85,0.75
2,chr1,4491370,4491376,chr1-4491370-4491376,0.38,0.28,0.49
3,chr1,4537659,4538001,chr1-4537659-4538001,0.78,0.68,0.89
4,chr1,5283176,5283324,chr1-5283176-5283324,0.71,0.53,0.74


In [6]:
# gene_df=  pd.read_csv('/data/ref/mouse/genome_anno/mm10_gene.bed', sep='\t', header=None)
# # extend 1 and 2 2000kb
# gene_df[1] = gene_df[1] - 2000
# gene_df[2] = gene_df[2] + 2000
# #gene_df.to_csv('mm10_gene_2kb.bed', sep='\t', header=None, index=None) 
# gene_df.head()

In [7]:
# use bedtools intersect to find the genes that overlap with the DMRs
command = f"bedtools intersect -a {dmr_dir}/{ct}.{_type}.aDMR.bed -b {home_dir}/mm10_gene_2kb.bed -wa -wb > {ct}.aDMR_gene.bed"
subprocess.run(command, shell=True)

CompletedProcess(args='bedtools intersect -a /data/mC_FILTER_DMR_BED/Astro-TE_NN.Total.aDMR.bed -b /home/qzeng_salk_edu/project/241018_pre_ml/mm10_gene_2kb.bed -wa -wb > Astro-TE_NN.aDMR_gene.bed', returncode=0)

In [8]:
dmr_gene_df = pd.read_csv(f'{ct}.aDMR_gene.bed', sep='\t', header=None)
# add a columns that is np.log2([6] - [4])
dmr_gene_df.columns = ['chr','start','end','dmr_id','2mo','9mo','18mo', 
                        'gene_chr','gene_start','gene_end','gene_id','gene_strand','gene_name','gene_type']
dmr_gene_df['old-young'] = dmr_gene_df['18mo'] -dmr_gene_df['2mo']
dmr_gene_df.head()

,chr,start,end,dmr_id,2mo,9mo,18mo,gene_chr,gene_start,gene_end,gene_id,gene_strand,gene_name,gene_type,old-young
0,chr1,4491370,4491376,chr1-4491370-4491376,0.38,0.28,0.49,chr1,4488931,4499354,ENSMUSG00000025902.9,-,Sox17,protein_coding,0.11
1,chr1,8411126,8411131,chr1-8411126-8411131,0.59,0.49,0.69,chr1,8359475,9301878,ENSMUSG00000025909.12,-,Sntg1,protein_coding,0.10
2,chr1,8565509,8565551,chr1-8565509-8565551,0.45,0.66,0.54,chr1,8359475,9301878,ENSMUSG00000025909.12,-,Sntg1,protein_coding,0.09
3,chr1,8778571,8779531,chr1-8778571-8779531,0.60,0.74,0.80,chr1,8359475,9301878,ENSMUSG00000025909.12,-,Sntg1,protein_coding,0.20
4,chr1,8939181,8939279,chr1-8939181-8939279,0.24,0.25,0.45,chr1,8359475,9301878,ENSMUSG00000025909.12,-,Sntg1,protein_coding,0.21


In [9]:

subprocess.run(f"rm {ct}.aDMR_gene.bed", shell=True)

CompletedProcess(args='rm Astro-TE_NN.aDMR_gene.bed', returncode=0)

In [10]:
# drop dmr_id, gen_id, gene_strand, old-young
dmr_gene_df['distance'] = np.minimum(abs(dmr_gene_df['start'] - dmr_gene_df['gene_start']), abs(dmr_gene_df['start'] - dmr_gene_df['gene_end']))
dmr_gene_df = dmr_gene_df.drop(columns=['chr','gene_chr','dmr_id','gene_id','gene_strand','old-young','gene_type'])
dmr_gene_df.head()

,start,end,2mo,9mo,18mo,gene_start,gene_end,gene_name,distance
0,4491370,4491376,0.38,0.28,0.49,4488931,4499354,Sox17,2439
1,8411126,8411131,0.59,0.49,0.69,8359475,9301878,Sntg1,51651
2,8565509,8565551,0.45,0.66,0.54,8359475,9301878,Sntg1,206034
3,8778571,8779531,0.60,0.74,0.80,8359475,9301878,Sntg1,419096
4,8939181,8939279,0.24,0.25,0.45,8359475,9301878,Sntg1,362697


In [11]:
dmr_gene_df.to_csv(f'{home_dir}/ml_input/{ct}/{ct}.aDMR_gene.csv', index=None)

## load enhancer

In [12]:
mc_abc_dir = '/data/female-amb/ABC.Enhancer/Re.ABC.By.Methylation'

In [13]:
mouse_genes = pd.read_csv(f"/home/qzeng_salk_edu/project/240321_abc_score/use.mouse_genes.csv", index_col =0)

In [14]:
def load_abc_df(ct, age):
    df = pd.read_csv(f'{mc_abc_dir}/{ct}/filtered.{ct}.{age}.abc_score.csv', index_col =0)
    if age == '8wk':
        age = '2mo'
    df.columns =  [f'{age}.activity',f'{age}.contact',f'{age}.abc_score']
    return df

In [15]:
young_abc_df = load_abc_df(ct, '8wk')
middle_abc_df = load_abc_df(ct, '9mo')
old_abc_df = load_abc_df(ct, '18mo')

In [16]:
# concat the three age groups
abc_df = pd.concat([young_abc_df, middle_abc_df, old_abc_df], axis=1)
#abc_df = abc_df.dropna()#
#abc_df = abc_df.fillna(0)

In [17]:
abc_df['chr'] = abc_df.index.map(lambda x: x.split('-')[0])
abc_df['start'] = abc_df.index.map(lambda x: int(x.split('-')[1]))
abc_df['end'] = abc_df.index.map(lambda x: int(x.split('-')[2]))
abc_df['gene_id'] = abc_df.index.map(lambda x: x.split('-')[3])
abc_df['gene_name'] = abc_df['gene_id'].map(lambda x: mouse_genes.loc[x,'gene_name'])

abc_df['gene_start'] = abc_df['gene_id'].map(lambda x: mouse_genes.loc[x,'start'])
abc_df['gene_end'] = abc_df['gene_id'].map(lambda x: mouse_genes.loc[x,'end'])
#abc_df['gene_length'] = abc_df['gene_end'] - abc_df['gene_start']
#abc_df['contact_distance'] = abs(abc_df['end'] - abc_df['start'])
# add eg_distance, based on the start is more close to gene_start ot gene_end; use the smaller absolute value
#abc_df['eg_distance'] = np.minimum(abs(abc_df['start'] - abc_df['gene_start']), abs(abc_df['start'] - abc_df['gene_end']))
#abc_df drop chr, start, end, gene_start, gene_end
#abc_df = abc_df.drop(columns=['chr','start','end','gene_start','gene_end'])
abc_df


,2mo.activity,2mo.contact,2mo.abc_score,9mo.activity,9mo.contact,9mo.abc_score,18mo.activity,18mo.contact,18mo.abc_score,chr,start,end,gene_id,gene_name,gene_start,gene_end
chr1-3377606-3377776-ENSMUSG00000051951,0.851065,14.0,0.026126,NaN,NaN,NaN,NaN,NaN,NaN,chr1,3377606,3377776,ENSMUSG00000051951,Xkr4,3205901,3671498
chr1-3427968-3428158-ENSMUSG00000051951,0.633718,19.0,0.026402,NaN,NaN,NaN,0.536358,27.0,0.027353,chr1,3427968,3428158,ENSMUSG00000051951,Xkr4,3205901,3671498
chr1-3466656-3466687-ENSMUSG00000051951,0.982034,10.0,0.021533,0.977607,14.0,0.028763,0.974313,13.0,0.023924,chr1,3466656,3466687,ENSMUSG00000051951,Xkr4,3205901,3671498
chr1-3515028-3515522-ENSMUSG00000051951,0.672663,21.0,0.030974,0.672572,16.0,0.022615,NaN,NaN,NaN,chr1,3515028,3515522,ENSMUSG00000051951,Xkr4,3205901,3671498
chr1-3540806-3540973-ENSMUSG00000051951,0.767304,12.0,0.020190,0.799047,13.0,0.021830,0.747663,25.0,0.035305,chr1,3540806,3540973,ENSMUSG00000051951,Xkr4,3205901,3671498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chr8-26975709-26976159-ENSMUSG00000109621,NaN,NaN,NaN,NaN,NaN,NaN,0.747725,42.0,0.025272,chr8,26975709,26976159,ENSMUSG00000109621,Gm45370,26996250,26998954
chr14-51885849-51886248-ENSMUSG00000115698,NaN,NaN,NaN,NaN,NaN,NaN,0.544710,98.0,0.021291,chr14,51885849,51886248,ENSMUSG00000115698,Gm49076,51926264,51929660
chr14-51927529-51928209-ENSMUSG00000115698,NaN,NaN,NaN,NaN,NaN,NaN,0.606250,104.0,0.025147,chr14,51927529,51928209,ENSMUSG00000115698,Gm49076,51926264,51929660
chr17-91747274-91747477-ENSMUSG00000117417,NaN,NaN,NaN,NaN,NaN,NaN,0.543962,17.0,0.023660,chr17,91747274,91747477,ENSMUSG00000117417,Gm49991,91566104,91628471


In [18]:
abc_df.to_csv(f'{home_dir}/ml_input/{ct}/{ct}.abc_enhancer.DMR_gene.csv', index=None)